In [9]:
# # Import required libraries
import PyPDF2
import re
import math
import string
import pandas as pd
import csv
from collections import Counter
WORD = re.compile(r"\w+")

In [2]:
#extract text form a pdf file
def getText(pdfReader):
    num_pages = pdfReader.numPages

    # Initialize a count for the number of pages
    count = 0

    # Initialize a text empty etring variable
    text = ''

    # Extract text from every page on the file
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText() 
        
    # Convert all strings to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+',' ',text)
    text = re.sub(r'\n',' ',text)
    text = re.sub(r',',' ',text)

    # Remove punctuation
    text = text.translate(str.maketrans('','',string.punctuation))
    return text

In [3]:
def getJobData(job):
    with open('jobdata.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        jobData = ''
        for row in csv_reader:
            if row["job"] == job:
                jobData = row["data"]
                break
                
    jobData = jobData.lower()

    # Remove numbers
    jobData = re.sub(r'\d+',' ',jobData)
    jobData = re.sub(r'\n',' ',jobData)
    jobData = jobData.translate(str.maketrans('','',string.punctuation))
    return jobData

In [4]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [5]:
def getScore(text1, text2): #get similarity using cosine similarity
    vec1 = text_to_vector(text1)
    vec2 = text_to_vector(text2)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [6]:
def intoDataset(name, pdfFileObj):
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    resumeText = getText(pdfReader)

    score = getScore(resumeText,jobData)
    df.loc[len(df.index)] = [name, job, resumeText, score] 

In [7]:
d = {'name': [], 'job':[], 'resumeData': [], 'score': []}
df = pd.DataFrame(d)

job = 'software developer'
jobData = getJobData(job)

name = 'Vivek'
pdfFileObj = open('Vivek-Resume.pdf','rb')
intoDataset(name,pdfFileObj)

name = 'Rajesh'
pdfFileObj = open('Vivek_Resume.pdf','rb')
intoDataset(name,pdfFileObj)

name = 'Robert'
pdfFileObj = open('Robert-resume.pdf','rb')
intoDataset(name,pdfFileObj)

name = 'Smith'
pdfFileObj = open('Smith-resume.pdf','rb')
intoDataset(name,pdfFileObj)

df

,name,job,resumeData,score
0,Vivek,software developer,vivek seeking a position to utilize my skills ...,0.316749
1,Rajesh,software developer,vivek phone no email kv gmailcom seeking a p...,0.244288
2,Robert,software developer,software developer ii robert smithphone ...,0.465833
3,Smith,software developer,robert smith embedded software developer phon...,0.446500


In [8]:
df.sort_values('score',ascending=False).head(10)    #top 10 candidates having highest resume score

,name,job,resumeData,score
2,Robert,software developer,software developer ii robert smithphone ...,0.465833
3,Smith,software developer,robert smith embedded software developer phon...,0.446500
0,Vivek,software developer,vivek seeking a position to utilize my skills ...,0.316749
1,Rajesh,software developer,vivek phone no email kv gmailcom seeking a p...,0.244288
